In [44]:
import tensorflow as tf
import numpy as np
import os
import cv2
from skimage.metrics import peak_signal_noise_ratio
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from google.colab.patches import cv2_imshow

def apply_median_filter(img):
    return cv2.medianBlur(img, 3)

def apply_mean_filter(img):
    return cv2.blur(img, (3, 3))

def apply_gaussian_filter(img):
    return cv2.GaussianBlur(img, (3, 3), 0)

def apply_bilateral_filter(img):
    return cv2.bilateralFilter(img, 5, 75, 75)

def get_psnr(original, processed):
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr

def add_poisson(img, lambda_val):
  poisson_noise = np.random.poisson(lambda_val, size=img.shape)
  noisy_image = img + poisson_noise
  if noisy_image.dtype != np.uint8:
    noisy_image = cv2.convertScaleAbs(noisy_image)
  return noisy_image

# Get the images in the pneumonia folder
data_folder = "/content/pneumonia/"
image_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.jpeg')]
images = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in image_files]

input_size = (124, 124)
images_resized = [cv2.resize(img, input_size) for img in images]
images_resized = np.array(images_resized)

train_images, test_images = train_test_split(images_resized, test_size=0.2, random_state=42)
train_images = tf.convert_to_tensor(train_images, dtype=tf.float32)
test_images = tf.convert_to_tensor(test_images, dtype=tf.float32)

lambda_values = [25, 50, 75]

noisy_images_list = []
for lambda_value in lambda_values:
    noisy_images = []
    for image in images_resized:
        noisy_image = add_poisson(image, lambda_value)
        noisy_images.append(noisy_image)
    noisy_images_list.append(noisy_images)

# Autoencoder
def create_autoencoder():
    input_img = Input(shape=(124, 124, 1))
    # Encoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    # Decoder
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')
    return autoencoder

average_psnr_values = {}
for idx, lambda_value in enumerate(lambda_values):
    autoencoder = create_autoencoder()
    noisy_images = np.array(noisy_images_list[idx])
    autoencoder.fit(noisy_images, images_resized, epochs=10, batch_size=10, shuffle=True, validation_split=0.2)

    psnr_values_autoencoder = []
    psnr_values_median = []
    psnr_values_mean = []
    psnr_values_gaussian = []
    psnr_values_bilateral = []

    for test_image in test_images:
        denoised_image_autoencoder = autoencoder.predict(test_image[np.newaxis, ...])[0, ...]
        psnr_autoencoder = get_psnr(test_image, denoised_image_autoencoder)
        psnr_values_autoencoder.append(psnr_autoencoder)

        noisy_image = test_image.numpy()
        denoised_image_median = apply_median_filter(noisy_image)
        denoised_image_mean = apply_mean_filter(noisy_image)
        denoised_image_gaussian = apply_gaussian_filter(noisy_image)
        denoised_image_bilateral = apply_bilateral_filter(noisy_image)

        psnr_median = get_psnr(test_image, denoised_image_median)
        psnr_mean = get_psnr(test_image, denoised_image_mean)
        psnr_gaussian = get_psnr(test_image, denoised_image_gaussian)
        psnr_bilateral = get_psnr(test_image, denoised_image_bilateral)

        psnr_values_median.append(psnr_median)
        psnr_values_mean.append(psnr_mean)
        psnr_values_gaussian.append(psnr_gaussian)
        psnr_values_bilateral.append(psnr_bilateral)

    average_psnr_values[lambda_value] = {
        "Autoencoder": np.mean(psnr_values_autoencoder),
        "Median Filter": np.mean(psnr_values_median),
        "Mean Filter": np.mean(psnr_values_mean),
        "Gaussian Filter": np.mean(psnr_values_gaussian),
        "Bilateral Filter": np.mean(psnr_values_bilateral)
    }

# Print average PSNR values for each lambda value
for lambda_value, psnr_values in average_psnr_values.items():
    print(f"Lambda value: {lambda_value}")
    print(f"Average PSNR values:")
    for method, average_psnr in psnr_values.items():
        print(f"{method}: {average_psnr:.2f}")

Exception ignored in: <function _xla_gc_callback at 0x7fb59dc9b760>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Epoch 1/10
8/8 [==============================] - 4s 255ms/step - loss: -1532.9872 - val_loss: -1970.6301
Epoch 2/10
8/8 [==============================] - 2s 259ms/step - loss: -1881.0621 - val_loss: -1978.2922
Epoch 3/10
8/8 [==============================] - 3s 341ms/step - loss: -1883.6188 - val_loss: -1979.0612
Epoch 4/10
8/8 [==============================] - 2s 226ms/step - loss: -1883.9084 - val_loss: -1979.2045
Epoch 5/10
8/8 [==============================] - 2s 226ms/step - loss: -1883.9739 - val_loss: -1979.2175
Epoch 6/10
8/8 [==============================] - 2s 227ms/step - loss: -1883.9791 - val_loss: -1979.2175
Epoch 7/10
8/8 [==============================] - 2s 225ms/step - loss: -1883.9792 - val_loss: -1979.2175
Epoch 8/10
8/8 [==============================] - 2s 232ms/step - loss: -1883.9792 - val_loss: -1979.2178
Epoch 9/10
8/8 [==============================] - 3s 350ms/step - loss: -1883.9792 - val_loss: -1979.2178
Epoch 10/10
1/1 [=============================

Lambda = 25 , Lambda = 50, Lambda = 75

Median Filter: 31.12, 31.12, 31.12

Mean Filter: 30.35, 30.35, 30.35

Bilateral Filter: 32.79, 32.79, 32.79

Gaussian Filter: 32.26, 32.26, 32.26

Autoencoder Model: 5.5, 5.5, 5.5

Answer the following questions:

a) Why is poisson distribution the ideal one to use to simulate noise for medical images? Why not gaussian or something else? Answer in terms of relevance of medical applications.

Medical images like X-rays and CT scans can be affected by specks of noise, making it harder to see important details. People can use "Poisson distribution" to understand this noise because it mimics how the image is created. This helps them develop better ways to remove the noise, like cleaning the specks off medical images, for better diagnosis and analysis. While other tools exist, they might not be as good at tackling this specific type of noise.

b) Which one performed the best? Why do you think this is the case?

The average PSNR values indicate that the Gaussian filter outperformed other denoising methods tested. This could be due to its ability to effectively smooth images while preserving their overall structure, which is crucial in medical imaging for accurate diagnosis. The Gaussian filter's adaptability to common Gaussian noise and its potential robustness across different noise levels may have contributed to its superior performance. Optimal parameter selection tailored to the noise characteristics and imaging application further enhances its effectiveness. The choice of the most suitable denoising method depends on various factors, including the imaging modality and the desired balance between noise reduction and fidelity.